## Pipeline de Collecte Multi-Sources (API REST & Web Scraping)

In [5]:
# scripts/make_finance_csvs_filled_2021_2025.py
import re
import time
import unicodedata
from pathlib import Path
from typing import Dict, Any, List, Tuple, Optional

import pandas as pd
import requests
from bs4 import BeautifulSoup  # <-- IMPORTANT (pip install beautifulsoup4)

SEASONS = [2021, 2022, 2023, 2024, 2025]

# Jolpica = Ergast-compatible (Ergast a été arrêté)
ERGAST = "https://api.jolpi.ca/ergast/f1"

OUT_DIR = Path("data")
OUT_DIR.mkdir(parents=True, exist_ok=True)

COST_CAP_BASELINE_USD = {
    2021: 145_000_000,
    2022: 140_000_000,
    2023: 135_000_000,
    2024: 135_000_000,
    2025: 135_000_000,
}

SALARY_SOURCES = {
    2021: "https://racingnews365.com/formula-1-driver-salaries-2021",
    2022: "https://racingnews365.com/what-are-the-driver-salaries-for-2022",
    2023: "https://racingnews365.com/f1-driver-salaries-2023",
    2024: "https://www.nbcnewyork.com/news/sports/how-much-are-f1-drivers-paid-salaries-2024/5192874/",
    2025: "https://racingnews365.com/2025-f1-driver-salaries-how-much-do-f1-drivers-earn",
}

NAME_OVERRIDES = {
    2025: {"bortoletto": "bortoleto"},
}

# ---------------------- HTTP (ROBUST) ----------------------

def get_json(url: str, retries: int = 6, backoff: float = 1.5) -> Dict[str, Any]:
    headers = {
        "User-Agent": "Mozilla/5.0 (compatible; F1PerformanceArchitect/1.0)",
        "Accept": "application/json",
    }
    last_err: Optional[Exception] = None
    for i in range(retries):
        try:
            r = requests.get(url, timeout=30, headers=headers)
            if r.status_code in (429, 403, 408, 441, 500, 502, 503, 504):
                time.sleep(backoff * (i + 1))
                continue
            r.raise_for_status()
            return r.json()
        except Exception as e:
            last_err = e
            time.sleep(backoff * (i + 1))
    raise last_err if last_err else RuntimeError(f"Échec requête: {url}")

def get_html(url: str) -> str:
    r = requests.get(url, timeout=30, headers={"User-Agent": "Mozilla/5.0"})
    r.raise_for_status()
    return r.text

# ---------------------- NORMALIZATION ----------------------

def normalize_text(s: str) -> str:
    s = str(s).strip()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = s.lower()
    s = re.sub(r"[^a-z0-9\s\-]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

# ---------------------- ERGAST-COMPATIBLE FETCH ----------------------

def get_constructor_standings(season: int) -> List[Dict[str, Any]]:
    url = f"{ERGAST}/{season}/constructorStandings.json"
    data = get_json(url)
    lists = data["MRData"]["StandingsTable"]["StandingsLists"]
    if not lists:
        return []
    standings = lists[0]["ConstructorStandings"]
    out = []
    for s in standings:
        c = s["Constructor"]
        out.append({
            "season": season,
            "constructorId": c["constructorId"],
            "constructorName": c["name"],
            "constructorRank": int(s["position"]),
            "constructorPoints": float(s["points"]),
        })
    return out

def get_driver_standings(season: int) -> List[Dict[str, Any]]:
    url = f"{ERGAST}/{season}/driverStandings.json"
    data = get_json(url)
    lists = data["MRData"]["StandingsTable"]["StandingsLists"]
    if not lists:
        return []
    standings = lists[0]["DriverStandings"]
    out = []
    for s in standings:
        d = s["Driver"]
        constructors = s.get("Constructors", [])
        constructor = constructors[0] if constructors else None
        out.append({
            "season": season,
            "driverId": d["driverId"],
            "givenName": d["givenName"],
            "familyName": d["familyName"],
            "driverName": f'{d["givenName"]} {d["familyName"]}',
            "driverCode": d.get("code", ""),
            "driverNumber": d.get("permanentNumber", ""),
            "driverRank": int(s["position"]),
            "driverPoints": float(s["points"]),
            "constructorId": constructor["constructorId"] if constructor else "",
            "constructorName": constructor["name"] if constructor else "",
        })
    return out

def spend_factor_from_rank(rank: int) -> float:
    if rank <= 3:
        return 0.98
    if rank <= 6:
        return 0.95
    return 0.92

# ---------------------- SALARY PARSING (NO LXML) ----------------------

def parse_salary_cell_to_millions(value: str) -> Tuple[Optional[float], Optional[float], Optional[float]]:
    if value is None:
        return (None, None, None)
    raw = str(value).strip()
    if raw == "" or raw.lower() in {"nan", "none"}:
        return (None, None, None)

    s = raw.lower().replace("$", "").replace("million", "").replace("m", "").strip()
    s = s.replace(",", ".")
    m = re.match(r"^\s*(\d+(\.\d+)?)\s*-\s*(\d+(\.\d+)?)\s*$", s)
    if m:
        a = float(m.group(1))
        b = float(m.group(3))
        return (a, b, (a + b) / 2.0)

    m = re.match(r"^\s*(\d+(\.\d+)?)\s*$", s)
    if m:
        a = float(m.group(1))
        return (a, a, a)

    m = re.search(r"(\d+(\.\d+)?)", s)
    if m:
        a = float(m.group(1))
        return (a, a, a)

    return (None, None, None)

def extract_salary_table_bs4(season: int, url: str) -> pd.DataFrame:
    """
    Extrait un tableau salaires avec BeautifulSoup (pas besoin de lxml).
    On cherche le tableau qui contient des en-têtes similaires à Driver/Name + Salary.
    """
    html = get_html(url)
    soup = BeautifulSoup(html, "html.parser")
    tables = soup.find_all("table")
    if not tables:
        raise ValueError(f"Aucun tableau HTML trouvé pour {season} sur {url}")

    def table_score(table) -> int:
        th_text = " ".join([normalize_text(th.get_text(" ")) for th in table.find_all("th")])
        score = 0
        if "driver" in th_text or "name" in th_text:
            score += 2
        if "salary" in th_text or "earn" in th_text:
            score += 2
        if "team" in th_text:
            score += 1
        return score

    best = max(tables, key=table_score)

    # headers
    headers = [h.get_text(" ", strip=True) for h in best.find_all("th")]
    if not headers:
        # parfois headers dans première ligne td
        first_row = best.find("tr")
        headers = [td.get_text(" ", strip=True) for td in first_row.find_all("td")]

    # rows
    rows = []
    for tr in best.find_all("tr"):
        cells = [td.get_text(" ", strip=True) for td in tr.find_all(["td", "th"])]
        if len(cells) < 2:
            continue
        rows.append(cells)

    # construire DF en mode flexible
    df = pd.DataFrame(rows[1:], columns=rows[0]) if len(rows) > 1 else pd.DataFrame()
    # détecter colonnes driver / salary
    cols_norm = {c: normalize_text(c) for c in df.columns}
    driver_col = None
    salary_col = None
    for c, nc in cols_norm.items():
        if driver_col is None and ("driver" in nc or "name" in nc):
            driver_col = c
        if salary_col is None and ("salary" in nc or "earn" in nc):
            salary_col = c

    if driver_col is None or salary_col is None:
        raise ValueError(f"Colonnes driver/salary introuvables pour {season}. Colonnes: {list(df.columns)}")

    out = pd.DataFrame({
        "season": season,
        "driver_raw": df[driver_col],
        "salary_raw": df[salary_col],
        "source_url": url
    })
    return out

def build_salary_lookup_by_driverid(season: int, drivers_df: pd.DataFrame, salary_table: pd.DataFrame) -> pd.DataFrame:
    drivers_df = drivers_df.copy()
    drivers_df["family_norm"] = drivers_df["familyName"].map(normalize_text)
    drivers_df["given_norm"] = drivers_df["givenName"].map(normalize_text)
    family_to_ids = drivers_df.groupby("family_norm")["driverId"].apply(list).to_dict()

    def map_row_to_driverid(name_raw: str) -> str:
        if not isinstance(name_raw, str):
            return ""
        n = normalize_text(name_raw)
        for wrong, correct in NAME_OVERRIDES.get(season, {}).items():
            n = re.sub(rf"\b{wrong}\b", correct, n)
        tokens = n.split()
        if not tokens:
            return ""
        family = tokens[-1]
        ids = family_to_ids.get(family, [])
        if len(ids) == 1:
            return ids[0]
        if len(tokens) >= 2:
            given = tokens[0]
            cand = drivers_df[(drivers_df["family_norm"] == family) & (drivers_df["given_norm"] == given)]
            if len(cand) == 1:
                return cand.iloc[0]["driverId"]
        return ""

    out = salary_table.copy()
    out["driverId"] = out["driver_raw"].apply(map_row_to_driverid)

    mins, maxs, mids = [], [], []
    for v in out["salary_raw"].tolist():
        mn, mx, md = parse_salary_cell_to_millions(v)
        mins.append(mn); maxs.append(mx); mids.append(md)

    out["salary_million_min"] = mins
    out["salary_million_max"] = maxs
    out["salary_million_mid"] = mids
    out["estimatedSalaryUSD"] = out["salary_million_mid"].apply(
        lambda x: int(round(x * 1_000_000)) if pd.notna(x) and x is not None else ""
    )
    out = out[out["driverId"] != ""].copy()
    return out[["season", "driverId", "estimatedSalaryUSD", "source_url"]]

# ---------------------- CSV BUILDERS ----------------------

def build_constructor_finance_csv() -> Path:
    rows = []
    for season in SEASONS:
        cap = COST_CAP_BASELINE_USD.get(season)
        standings = get_constructor_standings(season)
        for s in standings:
            factor = spend_factor_from_rank(s["constructorRank"])
            rows.append({
                "season": season,
                "constructorId": s["constructorId"],
                "constructorName": s["constructorName"],
                "constructorRank": s["constructorRank"],
                "constructorPoints": s["constructorPoints"],
                "costCapBaselineUSD": cap,
                "spendFactorProxy": factor,
                "estimatedSpendUSDProxy": round(cap * factor) if cap else "",
                "note": "Budget proxy basé sur Cost Cap + rang constructeur (budgets réels non publics)",
            })
        time.sleep(0.2)

    df = pd.DataFrame(rows).sort_values(["season", "constructorRank", "constructorId"])
    out_path = OUT_DIR / "f1_constructor_finance_2021_2025.csv"
    df.to_csv(out_path, index=False, encoding="utf-8")
    print(f"✅ {out_path} créé ({len(df)} lignes)")
    return out_path

def build_driver_finance_filled_csv() -> Path:
    all_rows = []
    for season in SEASONS:
        drivers = pd.DataFrame(get_driver_standings(season))
        if drivers.empty:
            continue

        src_url = SALARY_SOURCES[season]
        salary_table = extract_salary_table_bs4(season, src_url)
        salary_map = build_salary_lookup_by_driverid(season, drivers, salary_table)

        merged = drivers.merge(
            salary_map[["season", "driverId", "estimatedSalaryUSD", "source_url"]],
            on=["season", "driverId"],
            how="left",
        )

        merged["estimatedSalaryUSD"] = merged["estimatedSalaryUSD"].fillna("")
        merged["salarySourceURL"] = merged["source_url"].fillna("")
        merged.drop(columns=["source_url"], inplace=True)

        merged = merged[[
            "season", "driverId", "constructorId",
            "givenName", "familyName", "driverName",
            "driverCode", "driverNumber",
            "driverRank", "driverPoints",
            "estimatedSalaryUSD", "salarySourceURL",
        ]].copy()

        merged["note"] = "Salaire estimé (base) – hors bonus/sponsors – source publique"
        all_rows.append(merged)
        time.sleep(0.4)

    df_final = pd.concat(all_rows, ignore_index=True).sort_values(["season", "driverRank", "driverId"])
    out_path = OUT_DIR / "f1_driver_finance_2021_2025.csv"
    df_final.to_csv(out_path, index=False, encoding="utf-8")
    print(f"✅ {out_path} créé ({len(df_final)} lignes)")
    return out_path

if __name__ == "__main__":
    build_constructor_finance_csv()
    build_driver_finance_filled_csv()


✅ data\f1_constructor_finance_2021_2025.csv créé (50 lignes)
✅ data\f1_driver_finance_2021_2025.csv créé (110 lignes)


## Nettoyage des deux csv 

In [6]:
import pandas as pd

# 1) Constructeurs
df_c = pd.read_csv("data/f1_constructor_finance_2021_2025.csv")
df_c = df_c.drop(columns=["note"], errors="ignore")
df_c.to_csv("data/f1_constructor_finance_2021_2025_clean.csv", index=False)

# 2) Pilotes
df_d = pd.read_csv("data/f1_driver_finance_2021_2025.csv")

# supprimer colonnes de traçabilité (si tu veux)
df_d = df_d.drop(columns=["note", "salarySourceURL"], errors="ignore")

# convertir salaire en entier (en gardant les NaN possibles)
df_d["estimatedSalaryUSD"] = pd.to_numeric(df_d["estimatedSalaryUSD"], errors="coerce").astype("Int64")

df_d.to_csv("data/f1_driver_finance_2021_2025_clean.csv", index=False)

print("OK -> fichiers clean générés")


OK -> fichiers clean générés
